# Settings

## Constants

In [1]:
import os

PATH_DATA = './Dataset'
sensor_data = ['UserInfo.csv', 'Service.csv', 'ContextualFactor.csv', 'Interruptibility.csv']

# Utility Functions

In [2]:
import pandas as pd
import numpy as np
from collections import defaultdict
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import LeaveOneGroupOut
from imblearn.over_sampling import SMOTE
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier
from sklearn.svm import SVC
from sklearn.dummy import DummyClassifier
from sklearn.metrics import accuracy_score, f1_score


# Load the Dataset into Dataframe

In [3]:
dataframes = {
    filename: pd.read_csv(os.path.join(PATH_DATA, filename)).reset_index(drop=True)
    for filename in sensor_data
}
dfService = dataframes['Service.csv']
dfContextualFactor = dataframes['ContextualFactor.csv']
dfUserInfo = dataframes['UserInfo.csv']
dfInterruptibility = dataframes['Interruptibility.csv']

# Preprocessing

In [ ]:
# Select specific columns from dfContextualFactor, dfService, and dfUserInfo DataFrames
# Combine relevant columns to create a unified dataset
dfContextualFactor_selected_columns=dfContextualFactor[['uid','sid','activity1','activity2','activity3','userLocation','userPosition',]]
dfService_selected_columns=dfService[['weekOfExperiment','dayOfWeek','startTime', 'activityInquiry','availabilityInquiry','speechShadowing_1','speechShadowing_2','speechShadowing_3','speechShadowing_4','speechShadowing_5','continue-to-nextInquiry_1','continue-to-nextInquiry_2','continue-to-nextInquiry_3','continue-to-nextInquiry_4','endTime','endType']]
dfInterruptibility_selected_columns=dfInterruptibility[['SHORT_INTERACTION_interruptibility', 'LONG_INTERACTION_interruptibility']]

## Create binary columns for interaction types
## SHORT_INTERACTION: True if availabilityInquiry is not NaN
# dfCombinedAll['SHORT_INTERACTION_interruptibility'] = dfCombinedAll['availabilityInquiry'].notna()
## LONG_INTERACTION: True if continue-to-nextInquiry_1 is not NaN
# dfCombinedAll['LONG_INTERACTION_interruptibility'] = dfCombinedAll['continue-to-nextInquiry_1'].notna()

# ! For those who want to redefine LONG_INTERACTION with thresholds longer than 3 minutes,
# you can use the following columns:
# 5 minutes  => use continue-to-nextInquiry_2
# 7 minutes  => use continue-to-nextInquiry_3
# 9 minutes  => use continue-to-nextInquiry_4


dfCombinedAll=pd.concat([dfContextualFactor_selected_columns, dfService_selected_columns,dfInterruptibility_selected_columns], axis=1)

In [ ]:
# Concatenate and merge the all columns from dfUserInfo (dfUserInfo.csv)
dfUserInfo_selected_columns = dfUserInfo[['uid', 'homeType', 'speakerLocation', 'speakerPosition']]
dfCombinedAll = pd.merge(dfCombinedAll, dfUserInfo_selected_columns, on='uid', how='left')

## Position Processing

In [ ]:
# Define function to calculate proximity between user and speaker
def calculate_proximity(row):
    # Return 0 if user and speaker are in different rooms
    if row['userLocation'] != row['speakerLocation']:
        return 0
    # If in the same room
    elif row['userPosition'] == row['speakerPosition']:
        return 2  # Same position
    else:
        return 1  # Different positions (including missing position)

# Apply proximity calculation to create a new 'proximity' column
dfCombinedAll['proximity'] = dfCombinedAll.apply(calculate_proximity, axis=1)

print(dfCombinedAll[['userLocation', 'userPosition', 'speakerLocation', 'speakerPosition', 'proximity']].head(10))


      userRoom userPosition speakerRoom speakerPosition  proximity
0     Bed Room          Bed    Bed Room            Desk          1
1    Rest Room          NaN    Bed Room            Desk          0
2  Living Room          NaN    Bed Room            Desk          0
3  Living Room          NaN    Bed Room            Desk          0
4     Bed Room          Bed    Bed Room            Desk          1
5     Bed Room          Bed    Bed Room            Desk          1
6     Bed Room          Bed    Bed Room            Desk          1
7     Bed Room         Desk    Bed Room            Desk          2
8     Bed Room          Bed    Bed Room            Desk          1
9     Bed Room          Bed    Bed Room            Desk          1


## Activity and Time Processing

In [7]:
# Process activity columns for one-hot encoding
activity_cols = ['activity1', 'activity2', 'activity3']
df_activity = dfContextualFactor[activity_cols].copy()

# Get unique activities across all activity columns, excluding NaN
all_unique_activities = pd.unique(df_activity.values.ravel())
all_unique_activities = [x for x in all_unique_activities if pd.notna(x)]

# Create a DataFrame for one-hot encoding of activities
dfActivity_one_hot_encoding = pd.DataFrame(0, index=df_activity.index, columns=['act_' + str(val) for val in all_unique_activities])

# Perform one-hot encoding for each activity column
for col in activity_cols:
    for val in all_unique_activities:
        dfActivity_one_hot_encoding['act_' + str(val)] |= (df_activity[col] == val).astype(int)

# Concatenate one-hot encoded activity columns to dfCombinedAll
dfActivity_one_hot_encoding
dfCombinedAll = pd.concat([dfCombinedAll, dfActivity_one_hot_encoding], axis=1)

In [8]:
# Convert startTime to datetime and extract total minutes since midnight
dfCombinedAll['startTime'] = pd.to_datetime(dfCombinedAll['startTime'], format='%H:%M:%S', errors='coerce')
dfCombinedAll['minute'] = dfCombinedAll['startTime'].dt.hour * 60 + dfCombinedAll['startTime'].dt.minute

# Map days of the week to numerical values (MON=0, TUE=1, ..., SUN=6)
day_map = {'MON': 0, 'TUE': 1, 'WED': 2, 'THU': 3, 'FRI': 4, 'SAT': 5, 'SUN': 6}
dfCombinedAll['dayOfWeek'] = dfCombinedAll['dayOfWeek'].map(day_map)

# Bin minutes into 30-minute intervals for temporal analysis
dfCombinedAll['minute_bin'] = (dfCombinedAll['minute'] // 30).astype(int)

In [9]:
dfCombinedAll.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2901 entries, 0 to 2900
Data columns (total 43 columns):
 #   Column                                    Non-Null Count  Dtype         
---  ------                                    --------------  -----         
 0   uid                                       2901 non-null   int64         
 1   sid                                       2901 non-null   int64         
 2   activity1                                 2901 non-null   object        
 3   activity2                                 112 non-null    object        
 4   activity3                                 3 non-null      object        
 5   userRoom                                  2901 non-null   object        
 6   userPosition                              2414 non-null   object        
 7   weekOfExperiment                          2901 non-null   int64         
 8   dayOfWeek                                 2901 non-null   int64         
 9   startTime                     

# Feature Extraction

In [ ]:
# Select features for response prediction
dfFeatresForResponse = dfCombinedAll[['uid',
    'act_Taking a Nap / Sleeping','act_Hygiene','act_Eating','act_Using Media','act_Social Interaction',
    'act_Returning from Outside / Other Rooms','act_Studying / Working','act_Others','act_House Chores',
    'act_Self Caring','act_Visiting Outside / Other Rooms','act_Resting',
    'homeType','userLocation','userPosition','speakerLocation','speakerPosition',
    'minute_bin','dayOfWeek','SHORT_INTERACTION_interruptibility','LONG_INTERACTION_interruptibility']].copy()

# Encode categorical columns using LabelEncoder
categorical_columns = ['homeType', 'userLocation', 'userPosition', 'speakerLocation', 'speakerPosition', 'minute_bin']
label_encoders = defaultdict(LabelEncoder)

# Apply label encoding to each categorical column
for col in categorical_columns:
    dfFeatresForResponse[col] = label_encoders[col].fit_transform(dfFeatresForResponse[col])

# Create a copy of the encoded data for further processing
encoded_data = dfFeatresForResponse.copy()


encoded_data.info()
encoded_data.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2901 entries, 0 to 2900
Data columns (total 22 columns):
 #   Column                                    Non-Null Count  Dtype
---  ------                                    --------------  -----
 0   uid                                       2901 non-null   int64
 1   act_Taking a Nap / Sleeping               2901 non-null   int64
 2   act_Hygiene                               2901 non-null   int64
 3   act_Eating                                2901 non-null   int64
 4   act_Using Media                           2901 non-null   int64
 5   act_Social Interaction                    2901 non-null   int64
 6   act_Returning from Outside / Other Rooms  2901 non-null   int64
 7   act_Studying / Working                    2901 non-null   int64
 8   act_Others                                2901 non-null   int64
 9   act_House Chores                          2901 non-null   int64
 10  act_Self Caring                           2901 non-null   in

,uid,act_Taking a Nap / Sleeping,act_Hygiene,act_Eating,act_Using Media,act_Social Interaction,act_Returning from Outside / Other Rooms,act_Studying / Working,act_Others,act_House Chores,...,act_Resting,settingType,userRoom,userPosition,speakerRoom,speakerPosition,minute_bin,dayOfWeek,SHORT_INTERACTION_availability,LONG_INTERACTION_availability
0,1,1,0,0,0,0,0,0,0,0,...,0,0,1,0,0,1,12,0,True,False
1,1,0,1,0,0,0,0,0,0,0,...,0,0,7,3,0,1,15,0,False,False
2,1,0,0,1,0,0,0,0,0,0,...,0,0,5,3,0,1,12,1,False,False
3,1,0,0,1,0,0,0,0,0,0,...,0,0,5,3,0,1,13,1,False,False
4,1,1,0,0,0,0,0,0,0,0,...,0,0,1,0,0,1,18,1,True,False


## Label: SHORT_INTERACTION

In [ ]:
# Label distribution before balancing
print(encoded_data['SHORT_INTERACTION_interruptibility'].value_counts())

SHORT_INTERACTION_availability
True     2158
False     743
Name: count, dtype: int64


### Model Building and LOSO CV

In [ ]:
# Prepare features (X) and target variables (y) for SHORT_INTERACTION
X = encoded_data.drop(columns=['SHORT_INTERACTION_interruptibility', 'LONG_INTERACTION_interruptibility', 'uid'], axis=1) # Drop target and unrelated columns
y = encoded_data['SHORT_INTERACTION_interruptibility'] # Target variable
groups = encoded_data['uid'] # Group by user ID for Leave-One-Group-Out CV

# Initialize Leave-One-Group-Out cross-validator
logo = LeaveOneGroupOut()

# Initialize SMOTE for oversampling to handle class imbalance
smote = SMOTE(random_state=42)

# Define models to evaluate
models1 = {
    'Random Forest': RandomForestClassifier(n_estimators=100, max_depth=5, random_state=42),
    'Gradient Boosting': GradientBoostingClassifier(n_estimators=100, learning_rate=0.1, max_depth=5, random_state=42),
    'XGBoost': XGBClassifier(n_estimators=100, learning_rate=0.1, max_depth=5, random_state=42),
    'LightGBM': LGBMClassifier(n_estimators=100, learning_rate=0.1, max_depth=5, random_state=42),
    'CatBoost': CatBoostClassifier(iterations=100, depth=5, learning_rate=0.1, loss_function='Logloss', cat_features=[0], random_seed=42),
    'SVM': SVC(random_state=42),
    'Dummy': DummyClassifier(strategy="stratified", random_state=42)
}

results1_logo = {}

# Loop over each model
for model_name, model1 in models1.items():
    accuracies = []
    f1_scores = []

    # Loop over each fold in Leave-One-Group-Out cross-validation
    for i, (train_index, test_index) in enumerate(logo.split(X, y, groups)):
        # Split the data into training and testing sets for the current fold
        X_train, X_test = X.iloc[train_index], X.iloc[test_index]
        y_train, y_test = y.iloc[train_index], y.iloc[test_index]

        # Apply SMOTE to the training data to balance class distribution
        X_train_oversampled, y_train_oversampled = smote.fit_resample(X_train, y_train)

        # Train the model on the oversampled training data
        model1.fit(X_train_oversampled, y_train_oversampled.ravel())

        # Predict the target on the test data
        y_pred = model1.predict(X_test)

        # Evaluate the prediction
        accuracy = accuracy_score(y_test, y_pred)
        f1 = f1_score(y_test, y_pred, average='macro')

        accuracies.append(accuracy)
        f1_scores.append(f1)
    # Compute average accuracy and F1-score across all folds
    avg_accuracy = sum(accuracies) / len(accuracies)
    avg_f1_score = sum(f1_scores) / len(f1_scores)

    results1_logo[model_name] = {
        'Average Accuracy': avg_accuracy,
        'Average F1-Score (macro)': avg_f1_score
    }


C:\Users\Golibjon\AppData\Local\Temp\ipykernel_20060\3270779774.py:40: FutureWarning: Series.ravel is deprecated. The underlying array is already 1D, so ravel is not necessary.  Use `to_numpy()` for conversion to a numpy array instead.
  model1.fit(X_train_oversampled, y_train_oversampled.ravel())
C:\Users\Golibjon\AppData\Local\Temp\ipykernel_20060\3270779774.py:40: FutureWarning: Series.ravel is deprecated. The underlying array is already 1D, so ravel is not necessary.  Use `to_numpy()` for conversion to a numpy array instead.
  model1.fit(X_train_oversampled, y_train_oversampled.ravel())
C:\Users\Golibjon\AppData\Local\Temp\ipykernel_20060\3270779774.py:40: FutureWarning: Series.ravel is deprecated. The underlying array is already 1D, so ravel is not necessary.  Use `to_numpy()` for conversion to a numpy array instead.
  model1.fit(X_train_oversampled, y_train_oversampled.ravel())
C:\Users\Golibjon\AppData\Local\Temp\ipykernel_20060\3270779774.py:40: FutureWarning: Series.ravel is d

[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Number of positive: 2104, number of negative: 2104
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000308 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 93
[LightGBM] [Info] Number of data points in the train set: 4208, number of used features: 18
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with pos

C:\Users\Golibjon\AppData\Local\Temp\ipykernel_20060\3270779774.py:40: FutureWarning: Series.ravel is deprecated. The underlying array is already 1D, so ravel is not necessary.  Use `to_numpy()` for conversion to a numpy array instead.
  model1.fit(X_train_oversampled, y_train_oversampled.ravel())
C:\Users\Golibjon\AppData\Local\Temp\ipykernel_20060\3270779774.py:40: FutureWarning: Series.ravel is deprecated. The underlying array is already 1D, so ravel is not necessary.  Use `to_numpy()` for conversion to a numpy array instead.
  model1.fit(X_train_oversampled, y_train_oversampled.ravel())
C:\Users\Golibjon\AppData\Local\Temp\ipykernel_20060\3270779774.py:40: FutureWarning: Series.ravel is deprecated. The underlying array is already 1D, so ravel is not necessary.  Use `to_numpy()` for conversion to a numpy array instead.
  model1.fit(X_train_oversampled, y_train_oversampled.ravel())
C:\Users\Golibjon\AppData\Local\Temp\ipykernel_20060\3270779774.py:40: FutureWarning: Series.ravel is d

[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Number of positive: 2088, number of negative: 2088
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000267 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 93
[LightGBM] [Info] Number of data points in the train set: 4176, number of used features: 19
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with pos

C:\Users\Golibjon\AppData\Local\Temp\ipykernel_20060\3270779774.py:40: FutureWarning: Series.ravel is deprecated. The underlying array is already 1D, so ravel is not necessary.  Use `to_numpy()` for conversion to a numpy array instead.
  model1.fit(X_train_oversampled, y_train_oversampled.ravel())
C:\Users\Golibjon\AppData\Local\Temp\ipykernel_20060\3270779774.py:40: FutureWarning: Series.ravel is deprecated. The underlying array is already 1D, so ravel is not necessary.  Use `to_numpy()` for conversion to a numpy array instead.
  model1.fit(X_train_oversampled, y_train_oversampled.ravel())
C:\Users\Golibjon\AppData\Local\Temp\ipykernel_20060\3270779774.py:40: FutureWarning: Series.ravel is deprecated. The underlying array is already 1D, so ravel is not necessary.  Use `to_numpy()` for conversion to a numpy array instead.
  model1.fit(X_train_oversampled, y_train_oversampled.ravel())
C:\Users\Golibjon\AppData\Local\Temp\ipykernel_20060\3270779774.py:40: FutureWarning: Series.ravel is d

...

C:\Users\Golibjon\AppData\Local\Temp\ipykernel_20060\3270779774.py:40: FutureWarning: Series.ravel is deprecated. The underlying array is already 1D, so ravel is not necessary.  Use `to_numpy()` for conversion to a numpy array instead.
  model1.fit(X_train_oversampled, y_train_oversampled.ravel())
C:\Users\Golibjon\AppData\Local\Temp\ipykernel_20060\3270779774.py:40: FutureWarning: Series.ravel is deprecated. The underlying array is already 1D, so ravel is not necessary.  Use `to_numpy()` for conversion to a numpy array instead.
  model1.fit(X_train_oversampled, y_train_oversampled.ravel())


0:	learn: 0.6539924	total: 1.3ms	remaining: 128ms
1:	learn: 0.6247162	total: 2.38ms	remaining: 116ms
2:	learn: 0.5950981	total: 3.55ms	remaining: 115ms
3:	learn: 0.5743052	total: 4.72ms	remaining: 113ms
4:	learn: 0.5560487	total: 5.76ms	remaining: 109ms
5:	learn: 0.5401743	total: 6.8ms	remaining: 106ms
6:	learn: 0.5206125	total: 7.89ms	remaining: 105ms
7:	learn: 0.5042045	total: 8.98ms	remaining: 103ms
8:	learn: 0.4910786	total: 10.4ms	remaining: 106ms
9:	learn: 0.4796726	total: 11.8ms	remaining: 106ms
10:	learn: 0.4683951	total: 12.8ms	remaining: 104ms
11:	learn: 0.4590041	total: 14.1ms	remaining: 103ms
12:	learn: 0.4510206	total: 15.2ms	remaining: 102ms
13:	learn: 0.4427894	total: 16.3ms	remaining: 100ms
14:	learn: 0.4381560	total: 17.3ms	remaining: 98.3ms
15:	learn: 0.4324047	total: 18.6ms	remaining: 97.5ms
16:	learn: 0.4268654	total: 19.8ms	remaining: 96.4ms
17:	learn: 0.4219865	total: 20.8ms	remaining: 95ms
18:	learn: 0.4181830	total: 22.1ms	remaining: 94.3ms
19:	learn: 0.4126268	

C:\Users\Golibjon\AppData\Local\Temp\ipykernel_20060\3270779774.py:40: FutureWarning: Series.ravel is deprecated. The underlying array is already 1D, so ravel is not necessary.  Use `to_numpy()` for conversion to a numpy array instead.
  model1.fit(X_train_oversampled, y_train_oversampled.ravel())
C:\Users\Golibjon\AppData\Local\Temp\ipykernel_20060\3270779774.py:40: FutureWarning: Series.ravel is deprecated. The underlying array is already 1D, so ravel is not necessary.  Use `to_numpy()` for conversion to a numpy array instead.
  model1.fit(X_train_oversampled, y_train_oversampled.ravel())


34:	learn: 0.3826080	total: 51.4ms	remaining: 95.4ms
35:	learn: 0.3810853	total: 52.8ms	remaining: 93.8ms
36:	learn: 0.3793010	total: 54ms	remaining: 91.9ms
37:	learn: 0.3779164	total: 55.1ms	remaining: 89.9ms
38:	learn: 0.3767670	total: 56.3ms	remaining: 88.1ms
39:	learn: 0.3753033	total: 57.6ms	remaining: 86.4ms
40:	learn: 0.3737899	total: 58.8ms	remaining: 84.5ms
41:	learn: 0.3721286	total: 59.9ms	remaining: 82.8ms
42:	learn: 0.3713586	total: 61ms	remaining: 80.8ms
43:	learn: 0.3703218	total: 62.1ms	remaining: 79ms
44:	learn: 0.3695194	total: 63.4ms	remaining: 77.4ms
45:	learn: 0.3690117	total: 65.1ms	remaining: 76.4ms
46:	learn: 0.3681845	total: 66.7ms	remaining: 75.2ms
47:	learn: 0.3670547	total: 67.8ms	remaining: 73.5ms
48:	learn: 0.3655041	total: 69ms	remaining: 71.8ms
49:	learn: 0.3646003	total: 70.2ms	remaining: 70.2ms
50:	learn: 0.3633814	total: 71.5ms	remaining: 68.7ms
51:	learn: 0.3629180	total: 72.8ms	remaining: 67.2ms
52:	learn: 0.3618414	total: 74.1ms	remaining: 65.7ms
5

C:\Users\Golibjon\AppData\Local\Temp\ipykernel_20060\3270779774.py:40: FutureWarning: Series.ravel is deprecated. The underlying array is already 1D, so ravel is not necessary.  Use `to_numpy()` for conversion to a numpy array instead.
  model1.fit(X_train_oversampled, y_train_oversampled.ravel())
C:\Users\Golibjon\AppData\Local\Temp\ipykernel_20060\3270779774.py:40: FutureWarning: Series.ravel is deprecated. The underlying array is already 1D, so ravel is not necessary.  Use `to_numpy()` for conversion to a numpy array instead.
  model1.fit(X_train_oversampled, y_train_oversampled.ravel())
C:\Users\Golibjon\AppData\Local\Temp\ipykernel_20060\3270779774.py:40: FutureWarning: Series.ravel is deprecated. The underlying array is already 1D, so ravel is not necessary.  Use `to_numpy()` for conversion to a numpy array instead.
  model1.fit(X_train_oversampled, y_train_oversampled.ravel())
C:\Users\Golibjon\AppData\Local\Temp\ipykernel_20060\3270779774.py:40: FutureWarning: Series.ravel is d

### K-Fold CV

In [13]:
from sklearn.model_selection import KFold

# Initialize 5-fold cross-validator
kfold = KFold(n_splits=5, shuffle=False)

results1_kfold = {}

# Loop over each model
for model_name, model1 in models1.items():
    accuracies = []
    f1_scores = []

    # Loop over each fold in 5-fold cross-validation
    for train_index, test_index in kfold.split(X, y):
        # Split the data into training and testing sets for the current fold
        X_train, X_test = X.iloc[train_index], X.iloc[test_index]
        y_train, y_test = y.iloc[train_index], y.iloc[test_index]

        # Apply SMOTE to the training data to balance class distribution
        X_train_oversampled, y_train_oversampled = smote.fit_resample(X_train, y_train)

        # Train the model on the oversampled training data
        model1.fit(X_train_oversampled, y_train_oversampled.ravel())

        # Predict the target on the test data
        y_pred = model1.predict(X_test)

        # Evaluate the prediction
        accuracy = accuracy_score(y_test, y_pred)
        f1 = f1_score(y_test, y_pred, average='macro')

        accuracies.append(accuracy)
        f1_scores.append(f1)

    # Compute average accuracy and F1-score across all folds
    avg_accuracy = sum(accuracies) / len(accuracies)
    avg_f1_score = sum(f1_scores) / len(f1_scores)

    results1_kfold[model_name] = {
        'Average Accuracy': avg_accuracy,
        'Average F1-Score (macro)': avg_f1_score
    }


C:\Users\Golibjon\AppData\Local\Temp\ipykernel_20060\2766920084.py:23: FutureWarning: Series.ravel is deprecated. The underlying array is already 1D, so ravel is not necessary.  Use `to_numpy()` for conversion to a numpy array instead.
  model1.fit(X_train_oversampled, y_train_oversampled.ravel())
C:\Users\Golibjon\AppData\Local\Temp\ipykernel_20060\2766920084.py:23: FutureWarning: Series.ravel is deprecated. The underlying array is already 1D, so ravel is not necessary.  Use `to_numpy()` for conversion to a numpy array instead.
  model1.fit(X_train_oversampled, y_train_oversampled.ravel())
C:\Users\Golibjon\AppData\Local\Temp\ipykernel_20060\2766920084.py:23: FutureWarning: Series.ravel is deprecated. The underlying array is already 1D, so ravel is not necessary.  Use `to_numpy()` for conversion to a numpy array instead.
  model1.fit(X_train_oversampled, y_train_oversampled.ravel())
C:\Users\Golibjon\AppData\Local\Temp\ipykernel_20060\2766920084.py:23: FutureWarning: Series.ravel is d

[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Number of positive: 1716, number of negative: 1716
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000378 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 91
[LightGBM] [Info] Number of data points in the train set: 3432, number of used features: 18
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with pos

C:\Users\Golibjon\AppData\Local\Temp\ipykernel_20060\2766920084.py:23: FutureWarning: Series.ravel is deprecated. The underlying array is already 1D, so ravel is not necessary.  Use `to_numpy()` for conversion to a numpy array instead.
  model1.fit(X_train_oversampled, y_train_oversampled.ravel())
C:\Users\Golibjon\AppData\Local\Temp\ipykernel_20060\2766920084.py:23: FutureWarning: Series.ravel is deprecated. The underlying array is already 1D, so ravel is not necessary.  Use `to_numpy()` for conversion to a numpy array instead.
  model1.fit(X_train_oversampled, y_train_oversampled.ravel())
C:\Users\Golibjon\AppData\Local\Temp\ipykernel_20060\2766920084.py:23: FutureWarning: Series.ravel is deprecated. The underlying array is already 1D, so ravel is not necessary.  Use `to_numpy()` for conversion to a numpy array instead.
  model1.fit(X_train_oversampled, y_train_oversampled.ravel())


[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Number of positive: 1721, number of negative: 1721
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000279 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 93
[LightGBM] [Info] Number of data points in the train set: 3442, number of used features: 19
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with pos

C:\Users\Golibjon\AppData\Local\Temp\ipykernel_20060\2766920084.py:23: FutureWarning: Series.ravel is deprecated. The underlying array is already 1D, so ravel is not necessary.  Use `to_numpy()` for conversion to a numpy array instead.
  model1.fit(X_train_oversampled, y_train_oversampled.ravel())
C:\Users\Golibjon\AppData\Local\Temp\ipykernel_20060\2766920084.py:23: FutureWarning: Series.ravel is deprecated. The underlying array is already 1D, so ravel is not necessary.  Use `to_numpy()` for conversion to a numpy array instead.
  model1.fit(X_train_oversampled, y_train_oversampled.ravel())


0:	learn: 0.6571793	total: 913us	remaining: 90.5ms
1:	learn: 0.6208501	total: 1.73ms	remaining: 84.8ms
2:	learn: 0.5952402	total: 2.47ms	remaining: 80ms
3:	learn: 0.5726471	total: 3.23ms	remaining: 77.6ms
4:	learn: 0.5540803	total: 3.96ms	remaining: 75.3ms
5:	learn: 0.5352691	total: 4.68ms	remaining: 73.4ms
6:	learn: 0.5166234	total: 5.41ms	remaining: 71.9ms
7:	learn: 0.5021342	total: 6.17ms	remaining: 70.9ms
8:	learn: 0.4899828	total: 6.9ms	remaining: 69.8ms
9:	learn: 0.4802835	total: 7.72ms	remaining: 69.5ms
10:	learn: 0.4697036	total: 8.43ms	remaining: 68.2ms
11:	learn: 0.4628898	total: 9.22ms	remaining: 67.6ms
12:	learn: 0.4559652	total: 9.98ms	remaining: 66.8ms
13:	learn: 0.4509534	total: 10.8ms	remaining: 66.4ms
14:	learn: 0.4443255	total: 11.6ms	remaining: 66ms
15:	learn: 0.4377172	total: 12.5ms	remaining: 65.6ms
16:	learn: 0.4337399	total: 13.2ms	remaining: 64.7ms
17:	learn: 0.4299591	total: 14.2ms	remaining: 64.8ms
18:	learn: 0.4275821	total: 15.1ms	remaining: 64.3ms
19:	learn

C:\Users\Golibjon\AppData\Local\Temp\ipykernel_20060\2766920084.py:23: FutureWarning: Series.ravel is deprecated. The underlying array is already 1D, so ravel is not necessary.  Use `to_numpy()` for conversion to a numpy array instead.
  model1.fit(X_train_oversampled, y_train_oversampled.ravel())
C:\Users\Golibjon\AppData\Local\Temp\ipykernel_20060\2766920084.py:23: FutureWarning: Series.ravel is deprecated. The underlying array is already 1D, so ravel is not necessary.  Use `to_numpy()` for conversion to a numpy array instead.
  model1.fit(X_train_oversampled, y_train_oversampled.ravel())


90:	learn: 0.3392440	total: 73.3ms	remaining: 7.25ms
91:	learn: 0.3388139	total: 74.3ms	remaining: 6.46ms
92:	learn: 0.3383148	total: 75.1ms	remaining: 5.65ms
93:	learn: 0.3374711	total: 75.9ms	remaining: 4.85ms
94:	learn: 0.3370359	total: 76.8ms	remaining: 4.04ms
95:	learn: 0.3361340	total: 77.5ms	remaining: 3.23ms
96:	learn: 0.3354600	total: 78.3ms	remaining: 2.42ms
97:	learn: 0.3351031	total: 79.1ms	remaining: 1.61ms
98:	learn: 0.3345523	total: 79.9ms	remaining: 807us
99:	learn: 0.3337097	total: 80.8ms	remaining: 0us
0:	learn: 0.6526784	total: 1ms	remaining: 99ms
1:	learn: 0.6212271	total: 1.83ms	remaining: 89.6ms
2:	learn: 0.5944227	total: 2.61ms	remaining: 84.3ms
3:	learn: 0.5705365	total: 3.33ms	remaining: 80.1ms
4:	learn: 0.5500772	total: 4.17ms	remaining: 79.2ms
5:	learn: 0.5328618	total: 4.95ms	remaining: 77.6ms
6:	learn: 0.5191033	total: 5.77ms	remaining: 76.7ms
7:	learn: 0.5075404	total: 6.58ms	remaining: 75.7ms
8:	learn: 0.4937290	total: 7.37ms	remaining: 74.5ms
9:	learn: 0

C:\Users\Golibjon\AppData\Local\Temp\ipykernel_20060\2766920084.py:23: FutureWarning: Series.ravel is deprecated. The underlying array is already 1D, so ravel is not necessary.  Use `to_numpy()` for conversion to a numpy array instead.
  model1.fit(X_train_oversampled, y_train_oversampled.ravel())
C:\Users\Golibjon\AppData\Local\Temp\ipykernel_20060\2766920084.py:23: FutureWarning: Series.ravel is deprecated. The underlying array is already 1D, so ravel is not necessary.  Use `to_numpy()` for conversion to a numpy array instead.
  model1.fit(X_train_oversampled, y_train_oversampled.ravel())
C:\Users\Golibjon\AppData\Local\Temp\ipykernel_20060\2766920084.py:23: FutureWarning: Series.ravel is deprecated. The underlying array is already 1D, so ravel is not necessary.  Use `to_numpy()` for conversion to a numpy array instead.
  model1.fit(X_train_oversampled, y_train_oversampled.ravel())
C:\Users\Golibjon\AppData\Local\Temp\ipykernel_20060\2766920084.py:23: FutureWarning: Series.ravel is d

## Label: LONG_INTERACTION

In [ ]:
# Label distribution before balancing
print(encoded_data['LONG_INTERACTION_interruptibility'].value_counts())

LONG_INTERACTION_availability
False    1468
True     1433
Name: count, dtype: int64


### Model building and LOSO CV

In [ ]:
# Prepare features (X) and target variable (y) for predicting LONG_INTERACTION
X = encoded_data.drop(columns=['LONG_INTERACTION_interruptibility', 'SHORT_INTERACTION_interruptibility','uid'], axis=1)
y = encoded_data['LONG_INTERACTION_interruptibility']
groups = encoded_data['uid']

# Initialize Leave-One-Group-Out cross-validator
logo = LeaveOneGroupOut()

# Initialize SMOTE for oversampling to handle class imbalance
smote = SMOTE(random_state=42)

# Define models to evaluate
models2 = {
    'Random Forest': RandomForestClassifier(n_estimators=100, max_depth=5, random_state=42),
    'Gradient Boosting': GradientBoostingClassifier(n_estimators=100, learning_rate=0.1, max_depth=5, random_state=42),
    'XGBoost': XGBClassifier(n_estimators=100, learning_rate=0.1, max_depth=5, random_state=42),
    'LightGBM': LGBMClassifier(n_estimators=100, learning_rate=0.1, max_depth=5, random_state=42),
    'CatBoost': CatBoostClassifier(iterations=100, depth=5, learning_rate=0.1, loss_function='Logloss', cat_features=[0], random_seed=42),
    'SVM': SVC(random_state=42),
    'Dummy': DummyClassifier(strategy="stratified", random_state=42)
}

results2_logo = {}

# Loop over each model
for model_name, model2 in models2.items():
    accuracies = []
    f1_scores = []

    # Loop over each fold in Leave-One-Group-Out cross-validation
    for i, (train_index, test_index) in enumerate(logo.split(X, y, groups)):
        # Split the data into training and testing sets for the current fold
        X_train, X_test = X.iloc[train_index], X.iloc[test_index]
        y_train, y_test = y.iloc[train_index], y.iloc[test_index]

        # Apply SMOTE to the training data to balance class distribution
        X_train_oversampled, y_train_oversampled = smote.fit_resample(X_train, y_train)

        # Train the model on the oversampled training data
        model2.fit(X_train_oversampled, y_train_oversampled.ravel())

        # Predict the target on the test data
        y_pred = model2.predict(X_test)

        # Evaluate the prediction
        accuracy = accuracy_score(y_test, y_pred)
        f1 = f1_score(y_test, y_pred, average='macro')

        accuracies.append(accuracy)
        f1_scores.append(f1)

    # Compute average accuracy and F1-score across all folds
    avg_accuracy = sum(accuracies) / len(accuracies)
    avg_f1_score = sum(f1_scores) / len(f1_scores)

    results2_logo[model_name] = {
        'Average Accuracy': avg_accuracy,
        'Average F1-Score (macro)': avg_f1_score
    }

C:\Users\Golibjon\AppData\Local\Temp\ipykernel_20060\582202991.py:40: FutureWarning: Series.ravel is deprecated. The underlying array is already 1D, so ravel is not necessary.  Use `to_numpy()` for conversion to a numpy array instead.
  model2.fit(X_train_oversampled, y_train_oversampled.ravel())
C:\Users\Golibjon\AppData\Local\Temp\ipykernel_20060\582202991.py:40: FutureWarning: Series.ravel is deprecated. The underlying array is already 1D, so ravel is not necessary.  Use `to_numpy()` for conversion to a numpy array instead.
  model2.fit(X_train_oversampled, y_train_oversampled.ravel())
C:\Users\Golibjon\AppData\Local\Temp\ipykernel_20060\582202991.py:40: FutureWarning: Series.ravel is deprecated. The underlying array is already 1D, so ravel is not necessary.  Use `to_numpy()` for conversion to a numpy array instead.
  model2.fit(X_train_oversampled, y_train_oversampled.ravel())
C:\Users\Golibjon\AppData\Local\Temp\ipykernel_20060\582202991.py:40: FutureWarning: Series.ravel is depre

[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Number of positive: 1425, number of negative: 1425
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000317 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 90
[LightGBM] [Info] Number of data points in the train set: 2850, number of used features: 18
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with pos

C:\Users\Golibjon\AppData\Local\Temp\ipykernel_20060\582202991.py:40: FutureWarning: Series.ravel is deprecated. The underlying array is already 1D, so ravel is not necessary.  Use `to_numpy()` for conversion to a numpy array instead.
  model2.fit(X_train_oversampled, y_train_oversampled.ravel())
C:\Users\Golibjon\AppData\Local\Temp\ipykernel_20060\582202991.py:40: FutureWarning: Series.ravel is deprecated. The underlying array is already 1D, so ravel is not necessary.  Use `to_numpy()` for conversion to a numpy array instead.
  model2.fit(X_train_oversampled, y_train_oversampled.ravel())
C:\Users\Golibjon\AppData\Local\Temp\ipykernel_20060\582202991.py:40: FutureWarning: Series.ravel is deprecated. The underlying array is already 1D, so ravel is not necessary.  Use `to_numpy()` for conversion to a numpy array instead.
  model2.fit(X_train_oversampled, y_train_oversampled.ravel())


[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

C:\Users\Golibjon\AppData\Local\Temp\ipykernel_20060\582202991.py:40: FutureWarning: Series.ravel is deprecated. The underlying array is already 1D, so ravel is not necessary.  Use `to_numpy()` for conversion to a numpy array instead.
  model2.fit(X_train_oversampled, y_train_oversampled.ravel())
C:\Users\Golibjon\AppData\Local\Temp\ipykernel_20060\582202991.py:40: FutureWarning: Series.ravel is deprecated. The underlying array is already 1D, so ravel is not necessary.  Use `to_numpy()` for conversion to a numpy array instead.
  model2.fit(X_train_oversampled, y_train_oversampled.ravel())
C:\Users\Golibjon\AppData\Local\Temp\ipykernel_20060\582202991.py:40: FutureWarning: Series.ravel is deprecated. The underlying array is already 1D, so ravel is not necessary.  Use `to_numpy()` for conversion to a numpy array instead.
  model2.fit(X_train_oversampled, y_train_oversampled.ravel())


...

C:\Users\Golibjon\AppData\Local\Temp\ipykernel_20060\582202991.py:40: FutureWarning: Series.ravel is deprecated. The underlying array is already 1D, so ravel is not necessary.  Use `to_numpy()` for conversion to a numpy array instead.
  model2.fit(X_train_oversampled, y_train_oversampled.ravel())
C:\Users\Golibjon\AppData\Local\Temp\ipykernel_20060\582202991.py:40: FutureWarning: Series.ravel is deprecated. The underlying array is already 1D, so ravel is not necessary.  Use `to_numpy()` for conversion to a numpy array instead.
  model2.fit(X_train_oversampled, y_train_oversampled.ravel())


0:	learn: 0.6733803	total: 1.56ms	remaining: 155ms
1:	learn: 0.6561263	total: 2.58ms	remaining: 127ms
2:	learn: 0.6434857	total: 3.53ms	remaining: 114ms
3:	learn: 0.6338190	total: 4.55ms	remaining: 109ms
4:	learn: 0.6228912	total: 6.26ms	remaining: 119ms
5:	learn: 0.6144509	total: 8.09ms	remaining: 127ms
6:	learn: 0.6059800	total: 9.58ms	remaining: 127ms
7:	learn: 0.5980244	total: 10.5ms	remaining: 121ms
8:	learn: 0.5922831	total: 11.6ms	remaining: 117ms
9:	learn: 0.5868879	total: 12.5ms	remaining: 113ms
10:	learn: 0.5807013	total: 13.6ms	remaining: 110ms
11:	learn: 0.5769272	total: 14.7ms	remaining: 108ms
12:	learn: 0.5726039	total: 15.6ms	remaining: 105ms
13:	learn: 0.5694566	total: 17.1ms	remaining: 105ms
14:	learn: 0.5666418	total: 18.4ms	remaining: 105ms
15:	learn: 0.5630791	total: 19.6ms	remaining: 103ms
16:	learn: 0.5605786	total: 20.5ms	remaining: 100ms
17:	learn: 0.5579614	total: 22.1ms	remaining: 101ms
18:	learn: 0.5559744	total: 23.3ms	remaining: 99.5ms
19:	learn: 0.5540651	

C:\Users\Golibjon\AppData\Local\Temp\ipykernel_20060\582202991.py:40: FutureWarning: Series.ravel is deprecated. The underlying array is already 1D, so ravel is not necessary.  Use `to_numpy()` for conversion to a numpy array instead.
  model2.fit(X_train_oversampled, y_train_oversampled.ravel())


0:	learn: 0.6731870	total: 1.22ms	remaining: 121ms
1:	learn: 0.6599435	total: 2.59ms	remaining: 127ms
2:	learn: 0.6486434	total: 4.09ms	remaining: 132ms
3:	learn: 0.6348114	total: 5.44ms	remaining: 131ms
4:	learn: 0.6230028	total: 6.7ms	remaining: 127ms
5:	learn: 0.6146193	total: 7.92ms	remaining: 124ms
6:	learn: 0.6057363	total: 9.13ms	remaining: 121ms
7:	learn: 0.5975556	total: 10.5ms	remaining: 120ms
8:	learn: 0.5953176	total: 11ms	remaining: 111ms
9:	learn: 0.5894972	total: 12ms	remaining: 108ms
10:	learn: 0.5842878	total: 13ms	remaining: 105ms
11:	learn: 0.5789721	total: 14.1ms	remaining: 103ms
12:	learn: 0.5761572	total: 15ms	remaining: 101ms
13:	learn: 0.5717196	total: 16ms	remaining: 98.3ms
14:	learn: 0.5685181	total: 17.3ms	remaining: 98ms
15:	learn: 0.5657091	total: 18.7ms	remaining: 98.2ms
16:	learn: 0.5620395	total: 19.9ms	remaining: 97.3ms
17:	learn: 0.5588192	total: 20.9ms	remaining: 95ms
18:	learn: 0.5574453	total: 22ms	remaining: 93.6ms
19:	learn: 0.5555816	total: 23.1m

C:\Users\Golibjon\AppData\Local\Temp\ipykernel_20060\582202991.py:40: FutureWarning: Series.ravel is deprecated. The underlying array is already 1D, so ravel is not necessary.  Use `to_numpy()` for conversion to a numpy array instead.
  model2.fit(X_train_oversampled, y_train_oversampled.ravel())
C:\Users\Golibjon\AppData\Local\Temp\ipykernel_20060\582202991.py:40: FutureWarning: Series.ravel is deprecated. The underlying array is already 1D, so ravel is not necessary.  Use `to_numpy()` for conversion to a numpy array instead.
  model2.fit(X_train_oversampled, y_train_oversampled.ravel())
C:\Users\Golibjon\AppData\Local\Temp\ipykernel_20060\582202991.py:40: FutureWarning: Series.ravel is deprecated. The underlying array is already 1D, so ravel is not necessary.  Use `to_numpy()` for conversion to a numpy array instead.
  model2.fit(X_train_oversampled, y_train_oversampled.ravel())
C:\Users\Golibjon\AppData\Local\Temp\ipykernel_20060\582202991.py:40: FutureWarning: Series.ravel is depre

### K-Fold CV

In [16]:
from sklearn.model_selection import KFold

# Initialize 5-fold cross-validator
kfold = KFold(n_splits=5, shuffle=False)

results2_kfold = {}

# Loop over each model
for model_name, model2 in models2.items():
    accuracies = []
    f1_scores = []

    # Loop over each fold in 5-fold cross-validation
    for train_index, test_index in kfold.split(X, y):
        # Split the data into training and testing sets for the current fold
        X_train, X_test = X.iloc[train_index], X.iloc[test_index]
        y_train, y_test = y.iloc[train_index], y.iloc[test_index]

        # Apply SMOTE to the training data to balance class distribution
        X_train_oversampled, y_train_oversampled = smote.fit_resample(X_train, y_train)

        # Train the model on the oversampled training data
        model2.fit(X_train_oversampled, y_train_oversampled.ravel())

        # Predict the target on the test data
        y_pred = model2.predict(X_test)

        # Evaluate the prediction
        accuracy = accuracy_score(y_test, y_pred)
        f1 = f1_score(y_test, y_pred, average='macro')

        accuracies.append(accuracy)
        f1_scores.append(f1)

    # Compute average accuracy and F1-score across all folds
    avg_accuracy = sum(accuracies) / len(accuracies)
    avg_f1_score = sum(f1_scores) / len(f1_scores)

    results2_kfold[model_name] = {
        'Average Accuracy': avg_accuracy,
        'Average F1-Score (macro)': avg_f1_score
    }

C:\Users\Golibjon\AppData\Local\Temp\ipykernel_20060\1454448026.py:23: FutureWarning: Series.ravel is deprecated. The underlying array is already 1D, so ravel is not necessary.  Use `to_numpy()` for conversion to a numpy array instead.
  model2.fit(X_train_oversampled, y_train_oversampled.ravel())
C:\Users\Golibjon\AppData\Local\Temp\ipykernel_20060\1454448026.py:23: FutureWarning: Series.ravel is deprecated. The underlying array is already 1D, so ravel is not necessary.  Use `to_numpy()` for conversion to a numpy array instead.
  model2.fit(X_train_oversampled, y_train_oversampled.ravel())
C:\Users\Golibjon\AppData\Local\Temp\ipykernel_20060\1454448026.py:23: FutureWarning: Series.ravel is deprecated. The underlying array is already 1D, so ravel is not necessary.  Use `to_numpy()` for conversion to a numpy array instead.
  model2.fit(X_train_oversampled, y_train_oversampled.ravel())
C:\Users\Golibjon\AppData\Local\Temp\ipykernel_20060\1454448026.py:23: FutureWarning: Series.ravel is d

[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Number of positive: 1188, number of negative: 1188
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000282 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 88
[LightGBM] [Info] Number of data points in the train set: 2376, number of used features: 18
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with pos

C:\Users\Golibjon\AppData\Local\Temp\ipykernel_20060\1454448026.py:23: FutureWarning: Series.ravel is deprecated. The underlying array is already 1D, so ravel is not necessary.  Use `to_numpy()` for conversion to a numpy array instead.
  model2.fit(X_train_oversampled, y_train_oversampled.ravel())
C:\Users\Golibjon\AppData\Local\Temp\ipykernel_20060\1454448026.py:23: FutureWarning: Series.ravel is deprecated. The underlying array is already 1D, so ravel is not necessary.  Use `to_numpy()` for conversion to a numpy array instead.
  model2.fit(X_train_oversampled, y_train_oversampled.ravel())
C:\Users\Golibjon\AppData\Local\Temp\ipykernel_20060\1454448026.py:23: FutureWarning: Series.ravel is deprecated. The underlying array is already 1D, so ravel is not necessary.  Use `to_numpy()` for conversion to a numpy array instead.
  model2.fit(X_train_oversampled, y_train_oversampled.ravel())


[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

C:\Users\Golibjon\AppData\Local\Temp\ipykernel_20060\1454448026.py:23: FutureWarning: Series.ravel is deprecated. The underlying array is already 1D, so ravel is not necessary.  Use `to_numpy()` for conversion to a numpy array instead.
  model2.fit(X_train_oversampled, y_train_oversampled.ravel())
C:\Users\Golibjon\AppData\Local\Temp\ipykernel_20060\1454448026.py:23: FutureWarning: Series.ravel is deprecated. The underlying array is already 1D, so ravel is not necessary.  Use `to_numpy()` for conversion to a numpy array instead.
  model2.fit(X_train_oversampled, y_train_oversampled.ravel())


...

C:\Users\Golibjon\AppData\Local\Temp\ipykernel_20060\1454448026.py:23: FutureWarning: Series.ravel is deprecated. The underlying array is already 1D, so ravel is not necessary.  Use `to_numpy()` for conversion to a numpy array instead.
  model2.fit(X_train_oversampled, y_train_oversampled.ravel())
C:\Users\Golibjon\AppData\Local\Temp\ipykernel_20060\1454448026.py:23: FutureWarning: Series.ravel is deprecated. The underlying array is already 1D, so ravel is not necessary.  Use `to_numpy()` for conversion to a numpy array instead.
  model2.fit(X_train_oversampled, y_train_oversampled.ravel())


19:	learn: 0.5371649	total: 20ms	remaining: 79.8ms
20:	learn: 0.5356128	total: 21.1ms	remaining: 79.4ms
21:	learn: 0.5343331	total: 22.1ms	remaining: 78.5ms
22:	learn: 0.5325453	total: 23.2ms	remaining: 77.6ms
23:	learn: 0.5302191	total: 24.2ms	remaining: 76.6ms
24:	learn: 0.5280395	total: 25.3ms	remaining: 75.9ms
25:	learn: 0.5258492	total: 26.3ms	remaining: 74.9ms
26:	learn: 0.5235427	total: 27.6ms	remaining: 74.7ms
27:	learn: 0.5219664	total: 29ms	remaining: 74.6ms
28:	learn: 0.5199665	total: 30ms	remaining: 73.3ms
29:	learn: 0.5190939	total: 30.9ms	remaining: 72ms
30:	learn: 0.5178102	total: 32.1ms	remaining: 71.5ms
31:	learn: 0.5165753	total: 33.4ms	remaining: 71ms
32:	learn: 0.5155173	total: 34.5ms	remaining: 70.1ms
33:	learn: 0.5146989	total: 35.7ms	remaining: 69.2ms
34:	learn: 0.5136173	total: 36.7ms	remaining: 68.1ms
35:	learn: 0.5130749	total: 37.7ms	remaining: 67ms
36:	learn: 0.5124388	total: 38.6ms	remaining: 65.7ms
37:	learn: 0.5118897	total: 39.6ms	remaining: 64.6ms
38:	l

C:\Users\Golibjon\AppData\Local\Temp\ipykernel_20060\1454448026.py:23: FutureWarning: Series.ravel is deprecated. The underlying array is already 1D, so ravel is not necessary.  Use `to_numpy()` for conversion to a numpy array instead.
  model2.fit(X_train_oversampled, y_train_oversampled.ravel())
C:\Users\Golibjon\AppData\Local\Temp\ipykernel_20060\1454448026.py:23: FutureWarning: Series.ravel is deprecated. The underlying array is already 1D, so ravel is not necessary.  Use `to_numpy()` for conversion to a numpy array instead.
  model2.fit(X_train_oversampled, y_train_oversampled.ravel())


94:	learn: 0.4805022	total: 87.2ms	remaining: 4.59ms
95:	learn: 0.4799500	total: 88.1ms	remaining: 3.67ms
96:	learn: 0.4795702	total: 89.2ms	remaining: 2.76ms
97:	learn: 0.4791694	total: 90.1ms	remaining: 1.84ms
98:	learn: 0.4786992	total: 91ms	remaining: 919us
99:	learn: 0.4783280	total: 91.9ms	remaining: 0us
0:	learn: 0.6741379	total: 1.05ms	remaining: 104ms
1:	learn: 0.6571544	total: 2.01ms	remaining: 98.6ms
2:	learn: 0.6448336	total: 2.81ms	remaining: 90.8ms
3:	learn: 0.6332969	total: 3.65ms	remaining: 87.6ms
4:	learn: 0.6251638	total: 4.44ms	remaining: 84.4ms
5:	learn: 0.6178365	total: 5.4ms	remaining: 84.6ms
6:	learn: 0.6104293	total: 6.32ms	remaining: 83.9ms
7:	learn: 0.6044367	total: 7.12ms	remaining: 81.9ms
8:	learn: 0.5991271	total: 7.96ms	remaining: 80.5ms
9:	learn: 0.5935131	total: 8.78ms	remaining: 79ms
10:	learn: 0.5887796	total: 9.61ms	remaining: 77.8ms
11:	learn: 0.5843930	total: 10.4ms	remaining: 76.6ms
12:	learn: 0.5785885	total: 11.3ms	remaining: 75.8ms
13:	learn: 0.

C:\Users\Golibjon\AppData\Local\Temp\ipykernel_20060\1454448026.py:23: FutureWarning: Series.ravel is deprecated. The underlying array is already 1D, so ravel is not necessary.  Use `to_numpy()` for conversion to a numpy array instead.
  model2.fit(X_train_oversampled, y_train_oversampled.ravel())
C:\Users\Golibjon\AppData\Local\Temp\ipykernel_20060\1454448026.py:23: FutureWarning: Series.ravel is deprecated. The underlying array is already 1D, so ravel is not necessary.  Use `to_numpy()` for conversion to a numpy array instead.
  model2.fit(X_train_oversampled, y_train_oversampled.ravel())
C:\Users\Golibjon\AppData\Local\Temp\ipykernel_20060\1454448026.py:23: FutureWarning: Series.ravel is deprecated. The underlying array is already 1D, so ravel is not necessary.  Use `to_numpy()` for conversion to a numpy array instead.
  model2.fit(X_train_oversampled, y_train_oversampled.ravel())
C:\Users\Golibjon\AppData\Local\Temp\ipykernel_20060\1454448026.py:23: FutureWarning: Series.ravel is d

In [17]:
from tabulate import tabulate

# Models list
models = [
    'Random Forest', 'Gradient Boosting', 'XGBoost',
    'LightGBM', 'CatBoost', 'SVM', 'Dummy'
]

# Prepare table data
table_data = []
for model in models:
    row = [model]
    
    # Shell 1: SHORT_INTERACTION, Leave-One-Group-Out
    row.append(results1_logo.get(model, {}).get('Average Accuracy', 0.0))
    row.append(results1_logo.get(model, {}).get('Average F1-Score (macro)', 0.0))
    
    # Shell 3: LONG_INTERACTION, Leave-One-Group-Out
    row.append(results2_logo.get(model, {}).get('Average Accuracy', 0.0))
    row.append(results2_logo.get(model, {}).get('Average F1-Score (macro)', 0.0))

    # Shell 2: SHORT_INTERACTION, 5-fold
    row.append(results1_kfold.get(model, {}).get('Average Accuracy', 0.0))
    row.append(results1_kfold.get(model, {}).get('Average F1-Score (macro)', 0.0))
    
    # Shell 4: LONG_INTERACTION, 5-fold
    row.append(results2_kfold.get(model, {}).get('Average Accuracy', 0.0))
    row.append(results2_kfold.get(model, {}).get('Average F1-Score (macro)', 0.0))
    
    table_data.append(row)

# Define headers
headers = [
    'Model',
    'LOSO CV\nShort\nInteraction\nAccuracy', 'LOSO CV\nShort\nInteraction\nF1',
    'LOSO CV\nLong\nInteraction\nAccuracy', 'LOSO CV\nLong\nInteraction\nF1',
    '5-fold CV\nShort\nInteraction\nAccuracy', '5-fold CV\nShort\nInteraction\nF1',
    '5-fold CV\nLong\nInteraction\nAccuracy', '5-fold CV\nLong\nInteraction\nF1'
]

# Print the table
print("Machine learning model performance")
print(tabulate(table_data, headers=headers, tablefmt='fancy_grid', floatfmt='.4f', numalign="decimal"))


Machine learning model performance
╒═══════════════════╤═══════════════╤═══════════════╤═══════════════╤═══════════════╤═══════════════╤═══════════════╤═══════════════╤═══════════════╕
│ Model             │       LOSO CV │       LOSO CV │       LOSO CV │       LOSO CV │     5-fold CV │     5-fold CV │     5-fold CV │     5-fold CV │
│                   │         Short │         Short │          Long │          Long │         Short │         Short │          Long │          Long │
│                   │   Interaction │   Interaction │   Interaction │   Interaction │   Interaction │   Interaction │   Interaction │   Interaction │
│                   │      Accuracy │            F1 │      Accuracy │            F1 │      Accuracy │            F1 │      Accuracy │            F1 │
╞═══════════════════╪═══════════════╪═══════════════╪═══════════════╪═══════════════╪═══════════════╪═══════════════╪═══════════════╪═══════════════╡
│ Random Forest     │        0.8336 │        0.7385 │        0.68